# Project test

In this assignment, we are going to improve our codes in PattRecClasses and implement forward algorithm inside MarkovChain code as well as functions such as logprob and prob in Guassian in order to generate proper input values for forward algorithm.

In [1]:
from PattRecClasses import DiscreteD, GaussD, HMM, MarkovChain, Px_calc, gauss_logprob
from matplotlib import pyplot as plt
# For the code to work you might have to pip install scipy

import numpy as np